Finetuning eines GPT-3-Modell

In diesem Notebook wird dein Datensatz zusammengestellt, der ausschließlich zum Finetuning eines GPT-3-Modells dient

In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re
import json
import openai
import langdetect

# Datensatz erstellen für GPT3 Finetuning 

In [ ]:
# df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
# df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
# df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')


# MIT VIEL MEHR DATEN UND DANN DAFÜR DIE LOCATIONS RAUSSCHMEIßEN

df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_v2_2023-03-30_14-05-51-577.csv')
df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_v2_2023-04-05_08-52-36-619.csv')
df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_v2_2023-04-05_09-21-00-200.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

In [ ]:
# add a column genre restaurant to the df_rest
df_rest['genre'] = 'restaurant'
df_act['genre'] = 'activity'
df_hotel['genre'] = 'hotel'

Idee: nur den Text berücksichtigen. Noch kein Fokus auf zwei Modalitäten

In [ ]:
df_complete_ft = pd.concat([df_rest, df_act, df_hotel], axis=0)
print('All (raw): ', df_complete_ft.shape)
df_complete_ft = df_complete_ft.reset_index(drop=True)
df_complete_ft = df_complete_ft.drop_duplicates(subset=['reviewId'])
print('All (raw): ', df_complete_ft.shape)
df_complete_ft = df_complete_ft.dropna(subset=['text'])
print('All (raw): ', df_complete_ft.shape)
df_complete_ft['text'] = df_complete_ft['text'].astype(str)
print('All with text: ', df_complete_ft.shape)
# save 
df_complete_ft.to_csv('01_Raw_Data\dataset_für_weitere_forschung.csv', index=False)

In [ ]:
df_complete_ft = df_complete_ft[['text', 'reviewId', 'ur
l', 'placeId','categoryName', 'stars','title', 'reviewImageUrls/0']]
print('All with text: ', df_complete_ft.shape)

# drop rows with na in category name
df_complete_ft = df_complete_ft.dropna(subset=['categoryName'])
print('All with text: ', df_complete_ft.shape)

print('Unique categoryNames: ', df_complete_ft['categoryName'].nunique())
print('Unique categoryNames: ', df_complete_ft['categoryName'].unique())

print('Unique places: ', df_complete_ft['placeId'].nunique())

In [28]:
#  drop all rows with na in reviewImageUrls/0 in df_complete_ft

print('All with text: ', df_complete_ft.shape)
#df_complete_ft = df_complete_ft.dropna(subset=['reviewImageUrls/0'])
print('All with text: ', df_complete_ft.shape)

print('Unique categoryNames: ', df_complete_ft['categoryName'].nunique())
print('Unique categoryNames: ', df_complete_ft['categoryName'].unique())
# how many unique places are there in each category
print('Unique places: ', df_complete_ft['placeId'].nunique())


All with text:  (10137, 8)
All with text:  (10137, 8)
Unique categoryNames:  62
Unique categoryNames:  ['Mexican restaurant' 'American restaurant' 'Italian restaurant'
 'New American restaurant' 'German restaurant' 'Ethiopian restaurant'
 'Fast food restaurant' 'Family restaurant' 'Buffet restaurant'
 'Restaurant' 'Thai restaurant' 'Bar & grill' 'Seafood restaurant'
 'Sushi restaurant' 'Chinese restaurant' 'Ice cream shop'
 'Asian fusion restaurant' 'Vegetarian restaurant' 'French restaurant'
 'Bistro' 'Grill' 'Fine dining restaurant' 'Steak house'
 'Amusement center' 'Museum' 'Zoo' 'Bowling alley' 'Escape room center'
 'Tourist attraction' 'Movie theater' 'Miniature golf course'
 'Paintball center' 'Water park' 'Video arcade' 'Amusement park'
 'Tropical fish store' 'Aquarium' 'Exhibit' 'Boat rental service'
 'Theme park' 'Go-kart track' 'Bar' 'Recreation center'
 'Indoor playground' "Children's Amusement Centre" 'Skate park'
 'Public swimming pool' 'Amusement Centre' 'Golf course' 'Go

In [ ]:
# df_reduced = pd.read_csv('base_keywords_sentiment_reduced.csv')

# # drop all rows with na in reviewImageUrls/0
# print('All with text: ', df_reduced.shape)
# df_reduced = df_reduced.dropna(subset=['reviewImageUrls/0'])
# print('All with text: ', df_reduced.shape)


Jetzt. Andere Sprachen rausfiltern und Einträge die im Base-Datensatz sind rausfiltern. @TODO: RAUSFINDEN, OB AUF ANDERE LOCATOINS TRAINIEREN ODER DIE SELBEN WIE IM BASE_..._.csv Datensatz

In [ ]:
df = pd.read_csv ("finetuning_gpt3_v2.csv")
df.shape

In [ ]:
def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False

print('Starting...', df.shape)
df = df[df['text'].apply(is_english)]

print('Finsihed only English', df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english.csv', index=False)

In [ ]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod.csv")
print(df.shape)

df = df.groupby('placeId').head(25)
print(df['placeId'].nunique())

print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

In [ ]:
# Rows die END enthlaten rausnehmen, für COMPLETION PROMPT WICHTIG
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)
df = df[~df['text'].str.contains("END")]
print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

In [ ]:
df_reduced = pd.read_csv("base_keywords_sentiment_reduced.csv")
df_fine_tuning = pd.read_csv("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(f"Rows that appear in both dataframes:" , df_reduced['reviewId'].isin(df_fine_tuning['reviewId']).sum())

In [ ]:
def generate_prompt_gpt3(row):
    category = row["categoryName"]
    return f"A Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     category = row["categoryName"]
#     return f"Write a Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     stars = row["stars"]
#     category = row["categoryName"]
#     keywords = row["keywords_only"]
#     keywords = keywords[1:-1].split(",")[:3]
#     keywords = [k.strip() for k in keywords]
#     keywords = ", ".join(keywords)
#     keywords = keywords.replace("'", "")
#     return f"Write a {stars} stars Google Maps review about a {category}, with the following keywords: {keywords}."

In [ ]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)

df["prompt"] = df.apply(generate_prompt_gpt3, axis=1)
df["completion"] = df["text"]

print(df.shape)
print(df.columns)
print(df[['completion','prompt']].head(4999))

df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace_pc.csv', index=False)